# Create tensors sampled from Sentinel 2

This notebook generates a tensor set from set of points on a Sentinel satellite manifest. It is implemented using cuDF, cuPy and numba and requires a GPU to run.

The NVIDIA Docker container RAPIDS 21.02 was used to generate a computational environment.

In [ ]:
import cudf
from numba import cuda
import cupy as cp
import numpy as np
import gdal
import time
import csv

In [ ]:
# Disable memory pool for device memory (GPU)
cp.cuda.set_allocator(None)

In [ ]:
factor = 3
clip = False
suffix = str(factor)
if clip:
    suffix += 'c'
print("Factor:",factor,"clip:",clip,"suffix:",suffix)

In [ ]:
filename = 'onspd_sentinel_20210209_tiles.csv'

In [ ]:
%%time
df = cudf.read_csv(filename)

In [ ]:
# Convert filenames to categories for query
df['TCI_CAT'] = df['TCI_FILENAME'].astype('category').cat.codes

In [ ]:
df

In [ ]:
class cp_manifest:
    
    def __init__(self, src_filename):        # Instantiation - pass database
        self.tci_filename = src_filename
        self.nir_filename = src_filename.replace('TCI','B08')
        self.red_filename = src_filename.replace('TCI','B04')
        self.grn_filename = src_filename.replace('TCI','B03')
        self.blu_filename = src_filename.replace('TCI','B02')
        self.scl_filename = src_filename.replace('TCI_10m','SCL_20m')
        self.fci_filename = src_filename.replace('TCI','FCI').replace('jp2','TIFF')
        self.rbg_filename = src_filename.replace('TCI','RGB').replace('jp2','TIFF')
        self.nvi_filename = src_filename.replace('TCI','NVI').replace('jp2','TIFF')
        self.nvi_filename = src_filename.replace('TCI_10m','SCL_20m').replace('jp2','TIFF')
        self.npz_filename = src_filename.replace('TCI','NPZ').replace('jp2','npz')
        src_ds = gdal.Open(src_filename)
        self.metadata = src_ds.GetMetadata()
        self.geotransform = src_ds.GetGeoTransform()
        self.pixel_size = (self.geotransform[1], self.geotransform[5])
        self.tile_size = (src_ds.RasterXSize, src_ds.RasterYSize)
        self.origin = cp.array((self.geotransform[0], self.geotransform[3] + self.tile_size[1] * self.pixel_size[1]),dtype=cp.float64)
        src_ds = None

    def gen_opt_bands(self,type="uint8",red_threshold=4500,grn_threshold=4500,blu_threshold=4500,nir_threshold=6500):
        
        bands = []
        
        ds = gdal.Open(self.scl_filename)
        scl = cp.array(ds.GetRasterBand(1).ReadAsArray())
        scl = cp.array(cp.kron(scl,cp.ones((2,2))))
        ds = None
        mask = scl==4
        scl = None
        #print(mask,mask.shape,scl.shape)
        
        for i, file in enumerate((self.nir_filename,self.red_filename,self.grn_filename,self.blu_filename)):
            ds = gdal.Open(file)
            bands += [cp.asarray(ds.GetRasterBand(1).ReadAsArray())]
            
            scene_median = cp.median(bands[i][bands[i]>0])
            scene_mean = cp.mean(bands[i][bands[i]>0])
            scene_std = cp.std(bands[i][bands[i]>0])
            
            mask_median = cp.median(bands[i][mask])
            mask_mean = cp.mean(bands[i][mask])
            mask_std = cp.std(bands[i][mask])

            pc99_scene = cp.percentile(bands[i][bands[i]>0],99)
            pc99_mask = cp.percentile(bands[i][mask],99)
            pc991_scene = cp.percentile(bands[i][bands[i]>0],99.1)
            pc991_mask = cp.percentile(bands[i][mask],99.1)            
            pc999_scene = cp.percentile(bands[i][bands[i]>0],99.9)
            pc999_mask = cp.percentile(bands[i][mask],99.9)
           
            ds = None
            
            with open('spectral_analysis.csv', 'a') as x:
                csv_out = csv.writer(x)
                csv_out.writerow([file, i, scene_median, mask_median, scene_mean, mask_mean, scene_std, mask_std, pc99_scene, pc99_mask, pc991_scene, pc991_mask, pc999_scene, pc999_mask])
        
        self.bands = cp.stack(bands)
        
        start = time.time()
        self.channels =  cp.stack((cp.clip((self.bands[0]*(256/nir_threshold)),0,255).astype(cp.uint8),
                         cp.clip((self.bands[1]*(256/red_threshold)),0,255).astype(cp.uint8),
                         cp.clip((self.bands[2]*(256/grn_threshold)),0,255).astype(cp.uint8),
                         cp.clip((self.bands[3]*(256/blu_threshold)),0,255).astype(cp.uint8)))
        print("Computation time:",(time.time()-start)*1000,"ms")
        
        #self.ndvi_bands = (self.bands[0].astype(cp.float32) - self.bands[1].astype(cp.float32)) / (self.bands[0].astype(cp.float32) + self.bands[1].astype(cp.float32))
        
    def gen_auto_bands(self,factor,clip=False):
        
        bands = []
        band_factor = [1.0,factor,factor,factor]
        threshold = cp.zeros(4,dtype=cp.uint16)
        
        ds = gdal.Open(self.scl_filename)
        scl = cp.array(ds.GetRasterBand(1).ReadAsArray())
        scl = cp.array(cp.kron(scl,cp.ones((2,2))))
        ds = None
        mask = scl==4
        scl = None
        #print(mask,mask.shape,scl.shape)
        
        for i, file in enumerate((self.nir_filename,self.red_filename,self.grn_filename,self.blu_filename)):
            ds = gdal.Open(file)
            bands += [cp.asarray(ds.GetRasterBand(1).ReadAsArray())]

            threshold[i] = cp.percentile(bands[i][mask],99.5) * band_factor[i]
                
            print("Band:",i,"threshold:",threshold[i])
           
            ds = None
        
        self.bands = cp.stack(bands)
        
        start = time.time()
        
        if clip:
            self.channels =  cp.stack([cp.clip((self.bands[0].astype(cp.float32) / threshold[0]).astype(cp.float32),0,1),
                                       cp.clip((self.bands[1].astype(cp.float32) / threshold[1]).astype(cp.float32),0,1),
                                       cp.clip((self.bands[2].astype(cp.float32) / threshold[2]).astype(cp.float32),0,1),
                                       cp.clip((self.bands[3].astype(cp.float32) / threshold[3]).astype(cp.float32),0,1)])
        else:
            self.channels =  cp.stack([(self.bands[0].astype(cp.float32) / threshold[0]).astype(cp.float32),
                                       (self.bands[1].astype(cp.float32) / threshold[1]).astype(cp.float32),
                                       (self.bands[2].astype(cp.float32) / threshold[2]).astype(cp.float32),
                                       (self.bands[3].astype(cp.float32) / threshold[3]).astype(cp.float32)])            
        
        print("Computation time:",(time.time()-start)*1000,"ms")
        
        #self.ndvi_bands = (self.bands[0].astype(cp.float32) - self.bands[1].astype(cp.float32)) / (self.bands[0].astype(cp.float32) + self.bands[1].astype(cp.float32))

    def gen_ndvi_image(self,filename=None):
                
        if filename is None:
            filename = self.nvi_filename
            
        ramp = cp.stack((cp.hstack((cp.ones(128,dtype=cp.uint8)*255,cp.arange(255,0,-257/128).astype(cp.uint8))),
                         cp.hstack((cp.arange(0,255,2).astype(cp.uint8),cp.arange(255,127,-1).astype(cp.uint8))),
                         cp.zeros(256,dtype=cp.uint8)))

        src_ds = gdal.Open(src_filename)
        driver = gdal.GetDriverByName("GTiff")
        nvi_ds = driver.CreateCopy(filename, src_ds, 0)
        #print(nvi_ds)
        #print(self.channels.shape,ramp.shape,out_bands.shape)
        start = time.time()
        out_bands = self.gen_ndvi_bands(self.channels,ramp)
        print("Computation time:",(time.time()-start),"s")
        print(out_bands.shape)
        print(out_bands)
        
        for i in range(3):
            nvi_ds.GetRasterBand(i+1).WriteArray(cp.asnumpy(out_bands[i]))

        #print(out_bands[0])
        #print(self.bands[0])
        #print(self.bands[0])
        nvi_ds.FlushCache()
        nvi_ds = None
        src_ds = None
        
    def gen_ndvi_bands(self,channels,ramp):
        return ramp[:,cp.nan_to_num((((channels[0].astype(cp.float32) - channels[1].astype(cp.float32)) / 
                                      (channels[0].astype(cp.float32) + channels[1].astype(cp.float32))) / 
                                     2.0 + 0.5) * 255.0).astype(cp.uint8)]
    
    def save_bands(self):
        cp.savez_compressed(self.npz_filename,bands=self.bands)

In [ ]:
@cuda.jit('void(f8[:],f8[:],f8[:],f8,u2,f4[:,:,:],f4[:,:,:,:],u4[:])')
def gen_image_tensors(east,north,origin,scale,size,channels,tensors,seq):
    """ Generate a tensor set from set of points on a Sentinel satellite manifest .
    inputs
    east: UTM Easting Coordinates
    north: UTM Northing Coordinates
    origin: UTM Origin (tuple)
    scale: Coordinate units per pixel
    channels: Satellite colour channels (NIR, R, G, B)
    seq: offset in tensor file
    Outputs
    tensors: A set of tensors, one per point, size x size x 4
    Inputs
    ------
    {inputs}
    Outputs
    -------
    {outputs}
    """   
    i= cuda.grid(1)
    if i < east.size and i < north.size:
        # Calculate relative origins
        c_x = int(round(((east[i] - origin[0]) / scale) - (size / 2)))
        c_y = int(round(((north[i] - origin[1]) / scale) - (size / 2)))
        # Counters for all black or all white pixels
        height = channels.shape[1]
        for x in range(size):
            for y in range(size):
                for j in range(4):
                    if 0 <= height-(c_y+y)-1 < channels[j].shape[0] and 0 <= c_x+x < channels[j].shape[1]:
                        tensors[seq[i]][size-y-1][x][j] = channels[j][height-(c_y+y)][c_x+x]
                    else:
                        tensors[seq[i]][size-y-1][x][j] = 0

In [ ]:
%%time
img_size = 16
#tensors = cp.zeros((len(df['PCDS']),img_size,img_size,4),dtype=cp.uint8)
#tensors = cp.zeros((len(df['PCDS']),img_size,img_size,4),dtype=cp.float32)
tensors = cuda.mapped_array((len(df['PCDS']),img_size,img_size,4),dtype=np.float32)
#tensors[:] = np.zeros((len(df['PCDS']),img_size,img_size,4),dtype=np.float32)
cp_tensors = cp.asarray(tensors)

In [ ]:
print("Manifests:",df['TCI_CAT'].nunique())

In [ ]:
print(df['TCI_CAT'].unique().values_host)

In [ ]:
%%time
#with open('spectral_analysis.csv', 'w') as x:
#    csv_out = csv.writer(x)
#    csv_out.writerow(['FILE','BAND','SCENE_MEDIAN','MASK_MEDIAN','SCENE_MEAN','MASK_MEAN','SCENE_STD','MASK_STD','SCENE_99','MASK_99','SCENE_991','MASK_991','SCENE_999','MASK_999'])

for tci in df['TCI_CAT'].unique().values_host:
    query = 'TCI_CAT=='+str(tci)
    qdf = df.query(query)
    print(query)
    #print(qdf)
    src_filename = qdf['TCI_FILENAME'].iloc[0]
    print(tci,src_filename)
    manifest = cp_manifest(src_filename)
    #manifest.gen_opt_bands()
    manifest.gen_auto_bands(factor=factor,clip=clip)
    cuda.synchronize()
    #manifest.save_bands()
    #print(manifest.channels,manifest.channels.shape,manifest.origin,manifest.pixel_size[0],tensors.shape)
    #print(qdf['UTM_X'].shape,qdf['UTM_Y'].shape,manifest.origin,manifest.pixel_size[0],img_size,manifest.channels.shape,tensors.shape,qdf['SEQ'].shape)
    # Generate Keras Tensor Set
    gen_image_tensors.forall(len(qdf['PCDS']))(qdf['UTM_X'],qdf['UTM_Y'],manifest.origin,manifest.pixel_size[0],img_size,manifest.channels,tensors,qdf['SEQ'])
    cuda.synchronize()
    manifest = None

In [ ]:
!nvidia-smi

In [ ]:
%%time
cp.savez_compressed("sentinel_gb_995_"+suffix+"_20210317",tensors=cp_tensors)
print("File sentinel_gb_995_"+suffix+"_20210317.npz written")